# TensorBoard - LiveHandsOn

In diesem Jupyter-Notebook werden Beispiele zur Nutzung von TensorBoard gezeigt. Insbesondere wird auf die Themen

1. Darstellung von Metriken für Trainigs- und Validierungsdaten,
2. Anzeige des Modellgraphen und
3. Tuning von Hyperparametern

eingegangen. 

## Import der notwendigen Bibliotheken

In [ ]:
import os
import datetime

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import numpy as np
import matplotlib.pyplot as plt

## Datensatz

Im Folgenden wird der <a href='https://github.com/zalandoresearch/fashion-mnist'>Fashion-MNIST Datensatz</a> verwendet. Dieser besteht aus graustufen Bildern der Größe 28x28 Pixel. Ferner sind alle Bilder in eine von zehn Klassen eingeteilt.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Dazu kann ein Beispiel-Sample der Daten generiert werden.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

# Erstellung des Modells

Das Modell bekommt als Eingabe die Bilder als 28x28 Matrix und gibt eine Wahrscheinlichkeit für die einzelnen Labels (0-9) aus. Das Mapping der Labels ist

- 0 -> T-shirt/top, 
- 1 -> Trouser, 
- 2 -> Pullover, 
- 3 -> Dress, 
- 4 -> Coat,
- 5 -> Sandal, 
- 6 -> Shirt, 
- 7 -> Sneaker, 
- 8 -> Bag und
- 9 -> Ankle boot.

Anschließend wird das Modell kompiliert.

In [ ]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard-Callback
Innerhalb von Keras können Callbacks für Anweisungen zur Laufzeit verwendet werden. Dazu setzt man ein Directory für die Log-Dateien `log_dir`. Der Parameter `write_graph` dient zur Visualisierung des Berechnungsgraphen von dem Modell.

In [ ]:
log_dir = '/tf/tensorboard_log/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, write_graph=True)

## Logging eines Keras-Modells
Zum Training des werden zu Vorführzwecken lediglich die ersten 1000 Bilder der Trainingsdaten genutzt. Innerhalb eines Keras-Modells kann über die `fit`-Methode das TensorBoard-Callback über den Parameter `callbacks` beim Training übergeben werden.

In [ ]:
model.fit(x=train_images[:1000],
          y=train_labels[:1000],
          epochs=50,
          validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback],
          verbose=0)

# Logging eigener Kenngrößen

Zur Darstellung von selbst-erstellten Kenngrößen wird ein von TensorFlow bereitgestellter Writer für Log-Files  verwendet. Dieser wird mittels `tensorflow.summary.create_file_writer(LOGDIR)` erstellt. Mittels `tensorflow.summary.scalars` kann  die Darstellung im TensorBoard unter dem Tab `Scalars` geplottet werden.

In [ ]:
log_dir = '/tf/tensorboard_log/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = os.path.join(log_dir, 'my_scalar')
train_log_writer = tf.summary.create_file_writer(train_log_dir)

x_i = 0.
for (i, x) in zip(np.arange(50), np.random.normal(0, 1, 50)):
    with train_log_writer.as_default():
        x_i += (x+1)/2
        tf.summary.scalar('epoch_loss', x_i, step=i)     
        tf.summary.scalar('my_scalar', x_i, step=i)

# Analyse von Hyperparametern

Bei der Bearbeitung einer Aufgabe ist es oft vonnöten, verschiedene Modelle zu vergleichen. Insbesondere ist die Konfiguration eines Netzes ausschlaggebend für die Qualität einer Lösung. Zur Analyse mehrerer Parameterkonfigurationen kann die `HParams`-Funktion von Tensorboard genutzt werden. Solche Parameter können der verwendete Optimierer und derer Lernraten, verschiedene Netzkonfigurationen selbst, etc. sein.

## Definition der zu testenden Parameter

Die zu testenden Parameter können über die TensorBoard-Extension `hparams` definiert werden. Zum Schreiben des Log-Files für HParams wird die Konfiguration über die `hparams_config` an einen TensorFlow-Writer übergeben.

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([512, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0., 1.))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

## Erstellung des Modells

Zur zeitlichen Begrenzung der Laufzeit werden hier lediglich 1000 Samples der Trainingsdaten verwendet.

In [ ]:
def create_model(num_units, dropout_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_units, activation=tf.nn.relu),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    return model

def compile_and_fit(model, hparams, epochs, log_dir):
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_images[:1000],
        train_labels[:1000],
        epochs=epochs,
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir),
                   hp.KerasCallback(log_dir, hparams=hparams)],
        verbose=0
    )

## Training des Modells

Zum Test der Hyperparameter werden die Netze für die einzelnen Konfigurationen trainiert. Im Allgemeinen ist es sinnvoll, schon in den ersten Phasen der Bearbeitung einer Aufgabe mit dem Logging der Parameter zu beginnen. Der Vergleich kann somit über frühere Modelle geschehen. 

In [ ]:
for units in HP_NUM_UNITS.domain.values:
    for dropout_rate in np.random.random_sample(3):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer
            }
            
            print('Running example with\nunits: {}\ndropout_rate: {}\noptimizer: {}'
                  .format(units, dropout_rate, optimizer))
            
            log_dir = '/tf/tensorboard_log/hparam_tuning/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
            
            model = create_model(units, dropout_rate)
            _ = compile_and_fit(model, hparams, 10, log_dir)